Subset the data in the way to obtain better utility with the same attack strength.

In [3]:
import os
os.chdir("C:\\Users\\tsarcevic\\PycharmProjects\\fingerprinting-toolbox")
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pickle
import numpy as np
import pandas as pd
import collections

from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from scipy.stats import chi2
from imblearn.under_sampling import *

from datasets import GermanCredit
from attacks import *
from parameter_guidelines.guidelines import *

In [5]:
with open('parameter_guidelines/evaluation/german_credit/rel_horizontal_attack_utility_loss_gb_fpattr20.pickle', 'rb') as infile:
    utility_loss_20 = pickle.load(infile)

In [12]:
np.mean([np.mean(ut) for ut in utility_loss_20[1]])

-0.17047550469968925

In [15]:
[np.mean(ut) for ut in utility_loss_20[1]]

[-0.16551512417162523,
 -0.19166930571028606,
 -0.17783569503314728,
 -0.15610926180518256,
 -0.17162812157756382,
 -0.16194149497545085,
 -0.16703437969470697,
 -0.1719627167646843,
 -0.16639581873191425,
 -0.1746631285323312]

In [14]:
utility_loss_20[1] # --> 5 folds, 10 exp

[array([-0.30263158, -0.12025316, -0.17931034, -0.12080537, -0.10457516]),
 array([-0.31818182, -0.06      , -0.14788732, -0.12101911, -0.31125828]),
 array([-0.33757962, -0.0955414 , -0.25675676, -0.08391608, -0.11538462]),
 array([-0.33116883, -0.05960265, -0.17687075, -0.11486486, -0.09803922]),
 array([-0.36477987, -0.09090909, -0.16551724, -0.12080537, -0.11612903]),
 array([-0.30921053, -0.10322581, -0.18243243, -0.15483871, -0.06      ]),
 array([-0.30718954, -0.1025641 , -0.17687075, -0.13815789, -0.11038961]),
 array([-0.33974359, -0.10759494, -0.18791946, -0.13245033, -0.09210526]),
 array([-0.30718954, -0.10322581, -0.17931034, -0.11486486, -0.12738854]),
 array([-0.32692308, -0.12101911, -0.20134228, -0.12080537, -0.10322581])]

In [200]:
with open('parameter_guidelines/evaluation/german_credit/robustness_horizontal_universal_c95_ag05_fpattr20_e100.pickle', 'rb') as infile:
    robustness_20 = pickle.load(infile)

In [201]:
robustness_20

{1: 0.95,
 1.11: 0.9,
 1.25: 0.9,
 1.43: 0.9,
 1.67: 0.9,
 2: 0.9,
 2.5: 0.85,
 3: 0.85,
 4: 0.8,
 5: 0.8,
 6: 0.75,
 7: 0.7,
 8: 0.65,
 9: 0.65,
 10: 0.6,
 12: 0.5,
 15: 0.4,
 18: 0.25}

In [283]:
# adapted guidelines.py/attack_utility
def attack_outlier_removal_utility(model, data, target, ppf_q, n_folds=10): # ppf_q is equivalent to attack strength
    #data = data.to_numpy()
    X = data.drop(target, axis=1).to_numpy()
    y = data[target].to_numpy()
    #print(data.loc[(data['target']==2) & (data['duration']>2.1) & (data['duration']<2.3) & (data['checking_account']>-0.46)
    #              & (data['checking_account']<-0.45) & (data['credit_hist']<-2.3)])
    #print(data.iloc[915].to_numpy())
    #print(X)
    #print(y)

    # score = cross_val_score(model, X, y, cv=5)
    accuracy = []
    attack_strength = []
    for fold in range(n_folds):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=fold, shuffle=True)
        #print('####')
        #print(X_train[185])
        #print(y_train[185])
        # Covariance matrix
        covariance  = np.cov(X_train, rowvar=False)

        # Covariance matrix power of -1
        covariance_pm1 = np.linalg.matrix_power(covariance, -1)

        # Center point
        centerpoint = np.mean(X_train, axis=0)

        # Distances between center point and 
        distances = []
        for i, val in enumerate(X_train):
            p1 = val
            p2 = centerpoint
            distance = (p1-p2).T.dot(covariance_pm1).dot(p1-p2)
            distances.append(distance)
        distances = np.array(distances)

        # Cutoff (threshold) value from Chi-Sqaure Distribution for detecting outliers 
        cutoff = chi2.ppf(ppf_q, X_train.shape[1])   # <------ to reduce #outliers, increase the first parameter

        # Index of outliers
        outlierIndexes = np.where(distances > cutoff )

        print('--- Number of Outliers ----')
        attack_strength.append(len(outlierIndexes[0])/X_train.shape[0])
        print(str(len(outlierIndexes[0])) + " ({}%)".format(100*len(outlierIndexes[0])/X_train.shape[0]))

        print('--- Index of Outliers ----')
        print(outlierIndexes)
        # array([ 88, 185, 236, 266, 512, 520, 564, 602, 681, 682, 729, 761]

        #print('--- Observations found as outlier -----')
        #print(X_train[distances > cutoff , :])
        #print(y_train[distances > cutoff])
        #attacked_train = pd.concat([X_train[distances > cutoff , :], y_train[distances > cutoff]], axis=1)
        
        #print('--- Observations without outliers ------')
        #print(X_train[distances <= cutoff , :])
        #print(y_train[distances <= cutoff])
        attacked_X = X_train[distances <= cutoff , :]
        attacked_y = y_train[distances <= cutoff]

        model.fit(attacked_X, attacked_y)
        acc = accuracy_score(y_test, model.predict(X_test))
        accuracy.append(acc)
    return accuracy, attack_strength

In [395]:
fp_dir = "parameter_guidelines/fingerprinted_data/german_credit/attr_subset_20/"
fingerprinted_data = pd.read_csv(fp_dir + "universal_g{}_x1_l8_u1_sk0.csv".format(gamma))
data = GermanCredit().preprocessed(fp_data=fingerprinted_data)

attack_outlier_removal_utility(ppf_q=0.57, 
                               model=GradientBoostingClassifier(random_state=9), 
                               data=data, 
                               target='target', 
                               n_folds=5)

--- Number of Outliers ----
321 (40.125%)
--- Index of Outliers ----
(array([  0,   2,   4,   6,   7,   9,  10,  17,  18,  19,  21,  25,  27,
        28,  32,  34,  36,  38,  39,  40,  42,  46,  48,  51,  52,  54,
        57,  60,  63,  64,  67,  71,  74,  77,  78,  81,  82,  86,  88,
        96,  98, 100, 106, 108, 109, 110, 111, 114, 117, 128, 129, 130,
       137, 139, 144, 149, 151, 156, 158, 159, 161, 162, 163, 166, 169,
       170, 171, 172, 174, 175, 176, 179, 181, 182, 185, 197, 201, 206,
       209, 219, 220, 222, 225, 228, 229, 234, 236, 242, 246, 247, 248,
       251, 252, 254, 264, 266, 268, 269, 274, 275, 276, 277, 279, 282,
       283, 284, 289, 291, 292, 293, 294, 296, 299, 303, 304, 305, 306,
       312, 315, 316, 320, 323, 324, 325, 327, 333, 335, 337, 340, 346,
       348, 350, 351, 352, 355, 357, 358, 365, 366, 368, 370, 371, 376,
       385, 387, 388, 391, 392, 395, 398, 401, 403, 404, 406, 407, 408,
       410, 411, 412, 416, 417, 420, 421, 422, 424, 428, 438, 443,

([0.715, 0.755, 0.71, 0.73, 0.7],
 [0.40125, 0.39375, 0.39875, 0.39625, 0.40125])

In [321]:
outlier_attack_utilities = dict()  # dict(gamma:accuracies 5 fold); attack_strength = robustness_20[gamma]

In [353]:
np.mean([0.8525, 0.85125, 0.86, 0.845, 0.86625])

0.8549999999999999

In [318]:
np.mean([0.72, 0.75, 0.715, 0.74, 0.785])

0.742

In [398]:
outlier_attack_utilities[18] = [0.72, 0.75, 0.715, 0.74, 0.785]

In [406]:
outlier_attack_utilities = dict(collections.OrderedDict(sorted(outlier_attack_utilities.items())))

In [410]:
outlier_attack_utilities['baseline'] = outlier_attack_utilities[0]

In [413]:
outlier_attack_utilities

{1: [0.585, 0.685, 0.585, 0.56, 0.65],
 1.11: [0.64, 0.71, 0.635, 0.61, 0.645],
 1.25: [0.64, 0.71, 0.635, 0.61, 0.645],
 1.43: [0.64, 0.71, 0.635, 0.61, 0.645],
 1.67: [0.64, 0.71, 0.635, 0.61, 0.645],
 2: [0.64, 0.71, 0.635, 0.61, 0.645],
 2.5: [0.595, 0.685, 0.715, 0.65, 0.645],
 3: [0.595, 0.685, 0.715, 0.65, 0.645],
 4: [0.635, 0.69, 0.7, 0.68, 0.64],
 5: [0.635, 0.69, 0.7, 0.68, 0.64],
 6: [0.69, 0.7, 0.715, 0.69, 0.7],
 7: [0.675, 0.735, 0.69, 0.725, 0.7],
 8: [0.71, 0.765, 0.71, 0.7, 0.71],
 9: [0.71, 0.765, 0.71, 0.7, 0.71],
 10: [0.7, 0.755, 0.72, 0.675, 0.715],
 12: [0.74, 0.74, 0.715, 0.7, 0.76],
 15: [0.715, 0.755, 0.71, 0.73, 0.7],
 18: [0.72, 0.75, 0.715, 0.74, 0.785],
 'baseline': [0.73, 0.785, 0.745, 0.755, 0.765]}

In [419]:
(np.array(outlier_attack_utilities[18]) - np.array(outlier_attack_utilities['baseline'])) / np.array(outlier_attack_utilities['baseline'])

array([-0.01369863, -0.04458599, -0.04026846, -0.01986755,  0.02614379])

In [420]:
# TODO redo the baselines!!
baselines = dict()

In [414]:
with open('parameter_guidelines/evaluation/german_credit/outlier_attack_utilities_fpattr20_sk0.pkl', 'wb') as outfile:
    pickle.dump(outlier_attack_utilities, outfile)

In [426]:
# BASELINES
gamma = [1, 1.11, 1.25, 1.43, 1.67, 2, 2.5, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 18]
for g in gamma:
    fp_dir = "parameter_guidelines/fingerprinted_data/german_credit/attr_subset_20/"
    fingerprinted_data = pd.read_csv(fp_dir + "universal_g{}_x1_l8_u1_sk0.csv".format(g))
    data = GermanCredit().preprocessed(fp_data=fingerprinted_data)
    #data = data.to_numpy()
    X = data.drop('target', axis=1).to_numpy()
    y = data[target]

    # score = cross_val_score(model, X, y, cv=5)
    accuracy = []
    for fold in range(5):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=fold, shuffle=True)
        #X_train = X_train[:-750, :]
        #y_train = y_train[:-750]

        model = GradientBoostingClassifier(random_state=9)
        model.fit(X_train, y_train)
        acc = accuracy_score(y_test, model.predict(X_test))
        accuracy.append(acc)
    print(X_train.shape)
    print(accuracy)
    print(np.mean(accuracy))
    baselines[g] = accuracy

(800, 20)
[0.73, 0.785, 0.745, 0.755, 0.765]
0.756
(800, 20)
[0.75, 0.765, 0.715, 0.735, 0.76]
0.7449999999999999
(800, 20)
[0.775, 0.775, 0.715, 0.725, 0.76]
0.75
(800, 20)
[0.78, 0.765, 0.71, 0.73, 0.765]
0.75
(800, 20)
[0.765, 0.775, 0.73, 0.745, 0.735]
0.75
(800, 20)
[0.75, 0.765, 0.755, 0.765, 0.775]
0.762
(800, 20)
[0.785, 0.77, 0.715, 0.74, 0.73]
0.748
(800, 20)
[0.77, 0.79, 0.725, 0.74, 0.775]
0.76
(800, 20)
[0.775, 0.81, 0.735, 0.745, 0.75]
0.763
(800, 20)
[0.76, 0.775, 0.765, 0.755, 0.765]
0.764
(800, 20)
[0.755, 0.77, 0.72, 0.74, 0.745]
0.7460000000000001
(800, 20)
[0.775, 0.78, 0.76, 0.76, 0.75]
0.765
(800, 20)
[0.775, 0.78, 0.73, 0.72, 0.77]
0.755
(800, 20)
[0.775, 0.775, 0.74, 0.77, 0.775]
0.767
(800, 20)
[0.765, 0.765, 0.735, 0.765, 0.765]
0.7590000000000001
(800, 20)
[0.785, 0.78, 0.735, 0.72, 0.745]
0.7529999999999999
(800, 20)
[0.775, 0.77, 0.73, 0.76, 0.75]
0.757
(800, 20)
[0.76, 0.765, 0.745, 0.765, 0.755]
0.758


gamma=1 sk=0
(800, 20)
[0.73, 0.785, 0.745, 0.755, 0.765]
0.756 

In [433]:
rel_loss = dict()
for gamma in baselines:
    rel_loss[gamma] = (np.array(outlier_attack_utilities[gamma]) - np.array(baselines[gamma])) / np.array(outlier_attack_utilities[gamma])

In [434]:
rel_loss

{1: array([-0.24786325, -0.1459854 , -0.27350427, -0.34821429, -0.17692308]),
 1.11: array([-0.171875  , -0.07746479, -0.12598425, -0.20491803, -0.17829457]),
 1.25: array([-0.2109375 , -0.0915493 , -0.12598425, -0.18852459, -0.17829457]),
 1.43: array([-0.21875   , -0.07746479, -0.11811024, -0.19672131, -0.18604651]),
 1.67: array([-0.1953125 , -0.0915493 , -0.1496063 , -0.22131148, -0.13953488]),
 2: array([-0.171875  , -0.07746479, -0.18897638, -0.25409836, -0.20155039]),
 2.5: array([-0.31932773, -0.12408759,  0.        , -0.13846154, -0.13178295]),
 3: array([-0.29411765, -0.15328467, -0.01398601, -0.13846154, -0.20155039]),
 4: array([-0.22047244, -0.17391304, -0.05      , -0.09558824, -0.171875  ]),
 5: array([-0.19685039, -0.12318841, -0.09285714, -0.11029412, -0.1953125 ]),
 6: array([-0.0942029 , -0.1       , -0.00699301, -0.07246377, -0.06428571]),
 7: array([-0.14814815, -0.06122449, -0.10144928, -0.04827586, -0.07142857]),
 8: array([-0.0915493 , -0.01960784, -0.02816901, 

In [435]:
with open('parameter_guidelines/evaluation/german_credit/rel_outlier_attack_utilities_fpattr20_sk0.pkl', 'wb') as outfile:
    pickle.dump(rel_loss, outfile)

In [98]:
data = data.to_numpy()

In [99]:
# multivariate based on mahalonobis distance

# Covariance matrix
covariance  = np.cov(data , rowvar=False)

# Covariance matrix power of -1
covariance_pm1 = np.linalg.matrix_power(covariance, -1)

# Center point
centerpoint = np.mean(data , axis=0)

In [101]:
# Distances between center point and 
distances = []
for i, val in enumerate(data):
      p1 = val
      p2 = centerpoint
      distance = (p1-p2).T.dot(covariance_pm1).dot(p1-p2)
      distances.append(distance)
distances = np.array(distances)

# Cutoff (threshold) value from Chi-Sqaure Distribution for detecting outliers 
cutoff = chi2.ppf(0.99, data.shape[1])   # <------ to reduce #outliers, increase the first parameter

# Index of outliers
outlierIndexes = np.where(distances > cutoff )

print('--- Number of Outliers ----')
print(str(len(outlierIndexes[0])) + " ({}%)".format(100*len(outlierIndexes[0])/data.shape[0]))

print('--- Index of Outliers ----')
print(outlierIndexes)
# array([24, 35, 67, 81])

print('--- Observations found as outlier -----')
print(data[ distances > cutoff , :])

--- Number of Outliers ----
55 (5.5%)
--- Index of Outliers ----
(array([ 17,  22,  24,  65,  95, 108, 117, 138, 140, 151, 156, 173, 175,
       187, 197, 236, 237, 247, 264, 268, 290, 294, 297, 310, 314, 351,
       381, 413, 429, 438, 465, 494, 505, 516, 552, 597, 613, 646, 696,
       756, 763, 774, 818, 850, 859, 871, 889, 890, 892, 900, 906, 915,
       917, 950, 972], dtype=int64),)
--- Observations found as outlier -----
[[-1.25456565  0.75476341 -2.35086999 ... -0.82331789 -0.19601428
   1.        ]
 [-1.25456565 -0.90460432  1.34401408 ... -0.82331789  5.10166904
   1.        ]
 [ 1.13205258 -0.90460432  1.34401408 ... -0.82331789  5.10166904
   1.        ]
 ...
 [-1.25456565 -1.23647786 -0.50342796 ...  1.21459768 -0.19601428
   2.        ]
 [-0.45902624 -0.24085723  1.34401408 ...  1.21459768 -0.19601428
   1.        ]
 [-1.25456565  0.25695309 -1.42714898 ... -0.82331789 -0.19601428
   2.        ]]


In [42]:
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

NameError: name 'LinearRegression' is not defined

## Clustering approach

Targeted sub-sampling in a form of distribution-preserving subsampling. 
First, we cluster the data, then sample from the clusters based on their size. 
This can be experimented with a number of different cluster sizes. 
The goal is to obtain less accuracy loss by subsetting compared to random sampling.

In [10]:
# original data
data = GermanCredit().preprocessed()
X = data.drop('target', axis=1).to_numpy()
y = data['target']

In [23]:
# fingerprinted data
gamma=1
fp_dir = "parameter_guidelines/fingerprinted_data/german_credit/attr_subset_20/"
fingerprinted_data = pd.read_csv(fp_dir + "universal_g{}_x1_l8_u1_sk0.csv".format(gamma))
fp_data = GermanCredit().preprocessed(fp_data=fingerprinted_data)
#data = data.to_numpy()
X_fp = fp_data.drop('target', axis=1)#.to_numpy()
y_fp = fp_data['target']

In [34]:
# baseline
score_baseline = fp_cross_val_score(GradientBoostingClassifier(random_state=9), X, y, X_fp, y_fp, scoring='accuracy')['test_score']
score_baseline

array([0.8  , 0.735, 0.74 , 0.785, 0.745])

In [132]:
# this function will serve as a pipeline for cross validation attacked dataset
def attack_cross_val_score(X, y, target, attack, attack_strength, n_folds=5):
    accuracy = []
    for fold in range(n_folds):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=fold, shuffle=True)
        train = pd.concat([X_train, y_train], axis=1)
        attacked_train = attack.run(train, attack_strength, random_state=fold)
        attacked_X = attacked_train.drop(target, axis=1)
        attacked_y = attacked_train[target]
        model = GradientBoostingClassifier()
        model.fit(attacked_X, attacked_y)
        acc = accuracy_score(y_test, model.predict(X_test))
        accuracy.append(acc)
    return accuracy

In [174]:
# random attack
attack_utility_scores = attack_cross_val_score(X_fp, y_fp, 'target', HorizontalSubsetAttack(), 0.9)
attack_utility_scores

[0.675, 0.74, 0.695, 0.68, 0.645]

In [175]:
np.mean(attack_utility_scores)

0.687

### Targeted attack

In [12]:
def targeted_attack_cross_val_score(X, y, attack_strength, n_folds=5, sampler_version=1, sampler_n_neighbors=3):
    accuracy = []
    for fold in range(n_folds):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=fold, shuffle=True)
        train = pd.concat([X_train, y_train], axis=1)
        sampler = NearMiss(version=sampler_version, 
                           sampling_strategy=strategy(attack_strength, y_train), 
                           n_neighbors=sampler_n_neighbors)
        #sampler = InstanceHardnessThreshold(estimator=GradientBoostingClassifier(),
        #                                    sampling_strategy=strategy(attack_strength, y_train)) -- some internal error
        attacked_X, attacked_y = sampler.fit_resample(X_train, y_train)
        model = GradientBoostingClassifier()
        model.fit(attacked_X, attacked_y)
        acc = accuracy_score(y_test, model.predict(X_test))
        accuracy.append(acc)
    return accuracy

In [13]:
def strategy(attack_strength, y):
    strategy = y.value_counts() * (1-attack_strength)
    strategy = {key: round(x) for key, x in strategy.items()}
    return strategy

In [194]:
# targeted attack
attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.9)
attack_utility_scores
np.mean(attack_utility_scores) # version 1 ###### the best (and the only one actually better than random)

0.7050000000000001

In [197]:
# experimenting with the number of neighbors
res = dict()
for n in range(1, 10):
    # targeted attack
    attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.7, sampler_n_neighbors=n) # 3 shows to be the best (default)
    res[n] = np.mean(attack_utility_scores) # version 1 ###### the best (and the only one actually better than random)
print('version 1')
print(res)
print(max(res.values()))

version 1
{1: 0.6579999999999999, 2: 0.726, 3: 0.7230000000000001, 4: 0.73, 5: 0.7289999999999999, 6: 0.726, 7: 0.722, 8: 0.7089999999999999, 9: 0.716}
0.73


In [187]:
# targeted attack
attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.9, sampler_version=2)
attack_utility_scores
np.mean(attack_utility_scores) # version 2

[0.605, 0.69, 0.655, 0.65, 0.655]

In [190]:
# targeted attack
attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.9, sampler_version=3)
np.mean(attack_utility_scores) # version 3

0.6599999999999999

The final decision is the sampler NearMiss, version 1, with default number of neighbors, 3

In the following we create the experiment

In [7]:
robustness = dict()
for i in [4,8,12,16,20]:
    with open('parameter_guidelines/evaluation/german_credit/robustness_horizontal_universal_c95_ag05_fpattr{}_e100.pickle'.format(i), 'rb') as infile:
        robustness[i] = pickle.load(infile)

In [19]:
fpattr = 8
n_exp = 10
results = {gamma: [] for gamma in robustness[fpattr]}
for gamma, attack_strength in robustness[fpattr].items():
    print(gamma, attack_strength)
    for exp in range(n_exp):
        # take one fp ds
        fp_dir = "parameter_guidelines/fingerprinted_data/german_credit/attr_subset_{}/".format(fpattr)
        fingerprinted_data = pd.read_csv(fp_dir + "universal_g{}_x1_l8_u1_sk{}.csv".format(gamma, exp))
        fp_data = GermanCredit().preprocessed(fp_data=fingerprinted_data)
        #data = data.to_numpy()
        X_fp = fp_data.drop('target', axis=1)#.to_numpy()
        y_fp = fp_data['target']
        
        # calc baseline fp utility
        score_baseline = fp_cross_val_score(GradientBoostingClassifier(random_state=9), X, y, X_fp, y_fp, scoring='accuracy')['test_score']

        # calc attacked 
        attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, attack_strength)

        # take relative loss
        rel_loss = (score_baseline - attack_utility_scores) / score_baseline
        results[gamma].append(rel_loss)

1 0.9
1.11 0.9
1.25 0.9
1.43 0.9
1.67 0.9
2 0.9
2.5 0.85
3 0.85
4 0.8
5 0.8
6 0.75
7 0.7
8 0.65
9 0.65
10 0.6
12 0.5
15 0.4
18 0.25


In [208]:
results

{1: [array([0.1875    , 0.08163265, 0.08783784, 0.23566879, 0.1409396 ]),
  array([0.11538462, 0.125     , 0.24      , 0.22875817, 0.02758621]),
  array([0.18709677, 0.28205128, 0.13815789, 0.09803922, 0.21192053]),
  array([0.11643836, 0.2875817 , 0.31125828, 0.28571429, 0.26027397]),
  array([0.11184211, 0.05960265, 0.17763158, 0.14379085, 0.14285714]),
  array([0.06944444, 0.03896104, 0.24025974, 0.12666667, 0.14285714]),
  array([0.38709677, 0.15584416, 0.1986755 , 0.24358974, 0.40909091]),
  array([0.16556291, 0.13836478, 0.10596026, 0.2866242 , 0.31333333]),
  array([0.12337662, 0.08496732, 0.36842105, 0.19871795, 0.19620253]),
  array([0.19480519, 0.14012739, 0.12666667, 0.10810811, 0.09868421])],
 1.11: [array([0.0738255 , 0.03896104, 0.11486486, 0.13636364, 0.125     ]),
  array([0.05921053, 0.03289474, 0.16774194, 0.14012739, 0.02027027]),
  array([0.14465409, 0.11038961, 0.11764706, 0.12903226, 0.10810811]),
  array([0.03424658, 0.03246753, 0.08552632, 0.10666667, 0.09395973

In [20]:
with open('parameter_guidelines/evaluation/german_credit/rel_undersampling_attack_utilities_fpattr{}_e{}.pkl'.format(fpattr, n_exp), 'wb') as outfile:
    pickle.dump(results, outfile)

In [16]:
results[1]

[array([0.07142857, 0.00645161, 0.12582781, 0.1038961 , 0.02614379]),
 array([0.17006803, 0.01333333, 0.12666667, 0.13375796, 0.04794521]),
 array([0.03355705, 0.01324503, 0.14569536, 0.18867925, 0.11184211]),
 array([0.1       , 0.06451613, 0.07894737, 0.14285714, 0.20779221]),
 array([0.03378378, 0.04      , 0.21854305, 0.17721519, 0.29487179]),
 array([0.09803922, 0.01298701, 0.16666667, 0.15862069, 0.12337662]),
 array([0.07792208, 0.02580645, 0.12162162, 0.1025641 , 0.07189542]),
 array([0.11333333, 0.12258065, 0.15686275, 0.15584416, 0.22580645]),
 array([0.10625   , 0.05844156, 0.1       , 0.13664596, 0.08609272]),
 array([0.08441558, 0.08609272, 0.09210526, 0.10596026, 0.1038961 ])]